<a href="https://colab.research.google.com/github/youngchanseo/Hon.Gong.Machine-SNU/blob/main/(05_2_4)_%ED%99%95%EC%9D%B8%EB%AC%B8%EC%A0%9C_p262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
wine = pd.read_csv('https://raw.githubusercontent.com/youngchanseo/Hon.Gong.Machine-SNU/main/(05_2_0)_Raw%20data.csv')
#wine = pd.read_csv('htpps://bit.ly/wine_csv_data')

data = wine [['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

#훈련 세트의 입력 데이터와 target data를 train-input과 train_target 배열에 저장
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

# 모델을 만듦
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

# cross validatee ()이라는 교차 검증함수: 기본은 5폴드 교차 검증
# fit_time (모델을 훈련하는 시간), Score_time(모델을 검증하는 시간), test_score 키를 가진 딕셔너리 반환
from sklearn.model_selection import cross_validate
# scores = cross_validate(dt, train_input, train_target)

#타깃 클래스를 골고루 나누기 위해 StratifiedkFold 를 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())

#randint는 정수값을 뽑고 uniform은 실수값을 뽑음
from scipy.stats import uniform, randint

# 탐색할 매개변수의 범위
params = {'min_impurity_decrease': uniform(0.0001, 0.001), #0.0001에서 0.001사이의 실수값을 샘플링
          'max_depth': randint(20, 50), #20에서 50까지의 정수를 샘플링
          'min_samples_split': randint(2, 25),# 1에서 25까지의 정수를 샘플
          'min_samples_leaf': randint(1, 25),
          }
          # 샘플링 횟수: 사이킷런의 random search class인 RandomizedSearchCV의 n_iter 매개변수에 저장

마지막 Randomized Search CV예제에서 DecisionTreeClassifier 클래스에 splitter = 'random' 매개변수를 추가하고 다시 훈련해보세요 splitter 매개변수의 기본값은 'best'로 각 노드에서 최선의 분할을 찾습니다. 'random'이면 무작위로 분할한 다음 가장 좋은 것을 고름

In [5]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter = 'random',
                                               random_state=42), params,n_iter=100,
                         n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
#최적의 매개변수 조합 출력
print(gs.best_params_) #최적의 모델은 이미 전체 훈련 세트 (train_input, train_target)으로 훈련되어 best_estimator_속성에 저장되어 있음
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


Decision Tree의 node를 random하게 분할하기 때문에 100번의 반복에서 최적의 매개변수 조합을 찾지 못함.

평균 검증 점수와 테스트 점수가 모두 조금 낮음